In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets

from collections import Counter
from sklearn.model_selection import cross_val_score


# manual nested cross-validation for random forest on a classification dataset
from numpy import mean
from numpy import std

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the data
df = pd.read_csv('/content/drive/MyDrive/2023-CV&DL/Machine Learning/iris.csv')
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [ ]:
# Separate features and target  
X = df_feature=df.loc[:, ["sepal.length",	"sepal.width", "petal.length","petal.width"]]
y = df.loc[:, ["variety"]]

NOTE ON RANDOM STATE



In [ ]:
# configure the cross-validation procedure
outerloop = KFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in outerloop.split(X):
  print("train index", train_ix)
  print("test index", test_ix)

train index [  0   1   2   3   4   6   7   8   9  10  11  12  13  15  17  18  19  20
  21  22  23  24  25  26  27  28  30  31  32  33  34  35  36  37  38  39
  40  41  42  43  45  46  47  48  49  50  52  53  54  55  57  58  59  60
  61  62  63  64  65  66  67  68  69  70  71  72  73  74  76  77  79  80
  81  82  83  84  85  86  87  88  89  90  91  92  93  95  96  97  99 100
 101 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 121 122 123 124 125 126 127 128 129 130 132 133 134 135 136 137 138 139
 140 142 143 144 145 146 147 148 149]
test index [  5  14  16  29  44  51  56  75  78  94  98 102 120 131 141]
train index [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  20  21  22  23  24  25  26  27  28  29  30  32  34  36  37  38  39
  41  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59
  60  61  62  63  64  65  67  68  69  70  71  72  74  75  76  78  79  80
  81  82  83  85  86  87  88  89  91  93  94  95  96  97  98 1

In [ ]:
# configure the cross-validation procedure
outerloop = KFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in outerloop.split(X):
  print("train index", train_ix)
  print("test index", test_ix)

train index [  0   1   2   3   4   6   7   8   9  10  11  12  13  15  17  18  19  20
  21  22  23  24  25  26  27  28  30  31  32  33  34  35  36  37  38  39
  40  41  42  43  45  46  47  48  49  50  52  53  54  55  57  58  59  60
  61  62  63  64  65  66  67  68  69  70  71  72  73  74  76  77  79  80
  81  82  83  84  85  86  87  88  89  90  91  92  93  95  96  97  99 100
 101 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 121 122 123 124 125 126 127 128 129 130 132 133 134 135 136 137 138 139
 140 142 143 144 145 146 147 148 149]
test index [  5  14  16  29  44  51  56  75  78  94  98 102 120 131 141]
train index [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  20  21  22  23  24  25  26  27  28  29  30  32  34  36  37  38  39
  41  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59
  60  61  62  63  64  65  67  68  69  70  71  72  74  75  76  78  79  80
  81  82  83  85  86  87  88  89  91  93  94  95  96  97  98 1

END OF THE NOTE ON RANDOM STATE


*   If you use random_state=some_number, then you can guarantee that the output of Run 1 will be equal to the output of Run 2, i.e. your split will be always the same. 



In [ ]:
# configure the cross-validation procedure in the outer loop
outerloop = KFold(n_splits=10, shuffle=True, random_state=1)

# enumerate splits
outer_results = list()

for train_ix, test_ix in outerloop.split(X):
  #print("train index", train_ix)
  #print("test index", test_ix)
  
  # split train/test data
  X_train, X_test = X.loc[train_ix, :], X.loc[test_ix, :]
  y_train, y_test = y.loc[train_ix], y.loc[test_ix]
  
  # configure the cross-validation procedure in the inner loop
  innerloop = KFold(n_splits=3, shuffle=True, random_state=1)
  
  # define the model
  clf = RandomForestClassifier(random_state=1)
  
  # define search space
  grid = dict()
  grid['n_estimators'] = [10, 100, 500]
  grid['max_depth'] = [2, 4, 6]
  
  # define search for hyperparameters tuning 
  search = GridSearchCV(clf, grid, scoring='accuracy', cv=innerloop)
  
  # execute search
  result = search.fit(X_train, y_train)
  
  # get the best performing model (i.e., the one with the best hyperparameters)
  best_model = result.best_estimator_

  print("the best model is ", best_model)
  
  # evaluate model on the hold out dataset
  total = best_model.predict(X_test)
  
  # evaluate the model
  acc = accuracy_score(y_test, total)
  
  # store the result
  outer_results.append(acc)
  
  # report progress
  print('>>>Accuracy test=%.3f, Accuracy validation=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))

# summarize the estimated performance of the model
print('Global Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

the best model is  RandomForestClassifier(max_depth=4, n_estimators=10, random_state=1)
>>>Accuracy test=1.000, Accuracy validation=0.963, cfg={'max_depth': 4, 'n_estimators': 10}
the best model is  RandomForestClassifier(max_depth=2, random_state=1)
>>>Accuracy test=0.933, Accuracy validation=0.933, cfg={'max_depth': 2, 'n_estimators': 100}
the best model is  RandomForestClassifier(max_depth=4, n_estimators=10, random_state=1)
>>>Accuracy test=0.933, Accuracy validation=0.948, cfg={'max_depth': 4, 'n_estimators': 10}
the best model is  RandomForestClassifier(max_depth=2, random_state=1)
>>>Accuracy test=1.000, Accuracy validation=0.948, cfg={'max_depth': 2, 'n_estimators': 100}
the best model is  RandomForestClassifier(max_depth=4, random_state=1)
>>>Accuracy test=0.933, Accuracy validation=0.956, cfg={'max_depth': 4, 'n_estimators': 100}
the best model is  RandomForestClassifier(max_depth=4, random_state=1)
>>>Accuracy test=0.933, Accuracy validation=0.956, cfg={'max_depth': 4, 'n_es